In [1]:
#Redoing Figure 8 from the RDA paper (showcasing decadal changes in rainfall)

#Creates zonal mean of rainfall for each day and year from 1951-2007
#sorted by type - total, banded and local.

import numpy as np
import time
import os
import netCDF4 as nc

In [2]:
ferretpath = "/Users/Siwen/Desktop/ferret/bin"

In [3]:
rdapath = "APHRO_ZH_front_025deg_V1101."

In [4]:
years = np.arange(57) + 1951

In [5]:
Pchina_total = np.zeros((365,80,57))
Pchina_band = np.zeros((365,80,57))
Pchina_local = np.zeros((365,80,57))

xmin=21
xmax=72
ymin=1
ymax=80

In [7]:
for i in years:
    
    #Build filename
    filename = ferretpath + '/' + rdapath + str(i) + '.nc'
    print(filename)
    myf = nc.Dataset(filename, 'r')
    
    #Obtain fraction of different types of rainfall
    P_total =  myf.variables['P'][:]
    P_band1 = myf.variables['P_band1'][:]
    P_band2 = myf.variables['P_band2'][:]
    P_band = P_band1 + P_band2
    P_local = myf.variables['P_leftover'][:]
    
    #Find seasonal amounts of each type of rainfall
    yr=i-1951
    
    #leaving out day 366 in leap years for convenience
    Pchina_total[:,:,yr]=np.nanmean(P_total[0:365,ymin-1:ymax,xmin-1:xmax], axis=2)
    Pchina_band[:,:,yr]=np.nanmean(P_band[0:365,ymin-1:ymax,xmin-1:xmax], axis=2)
    Pchina_local[:,:,yr]=np.nanmean(P_local[0:365,ymin-1:ymax,xmin-1:xmax], axis=2)
            
    myf.close()

/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1951.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1952.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1953.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1954.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1955.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1956.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1957.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1958.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1959.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1960.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1961.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1962.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1963.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_025deg_V1101.1964.nc
/Users/Siwen/Desktop/ferret/bin/APHRO_ZH_front_0

In [9]:
outfilename = "Pchina_type.nc"

if os.path.exists(outfilename):
    os.remove(outfilename)
    
if os.path.exists("/Users/Siwen/Desktop/ferret/bin/" + outfilename):
    os.remove("/Users/Siwen/Desktop/ferret/bin/" + outfilename)

#Save as netCDF file
outfile = nc.Dataset(outfilename, "w", format="NETCDF4")

In [10]:
#create dimensions
days_dim = outfile.createDimension('days', None)
lat_dim = outfile.createDimension('lat', 80)
yr_dim = outfile.createDimension('years', 57)

dim_tuple = ("days","lat","years")

#create variables inside netCDF file
Pchina_total_out = outfile.createVariable("Pchina_total", 'f8', dim_tuple)
Pchina_band_out = outfile.createVariable("Pchina_band", 'f8', dim_tuple)
Pchina_local_out = outfile.createVariable("Pchina_local",'f8', dim_tuple)


In [11]:
outfile.variables

Pchina_total_out[:] = Pchina_total
Pchina_band_out[:] = Pchina_band
Pchina_local_out[:] = Pchina_local

outfile.close()
os.rename("./" + outfilename, "/Users/Siwen/Desktop/ferret/bin/" + outfilename)